# Spatial convergence test

In [41]:
from pathlib import Path
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests

pd.options.display.max_columns = 5

In [42]:
headers = {
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
}
print("Get existing data from gist")
response = requests.get(
    f"https://api.github.com/gists/73fa6531f28da2b3633a7ddaca38a7cd",
    headers=headers,
)
data = json.loads(response.json()["files"]["convergence_test.json"]["content"])

Get existing data from gist


In [43]:
runs = set()
results = defaultdict(list)
spatial_convergence = []
for git_hash, d in data.items():
    runs |= set(d.keys())
    if "dx0.1_dt0.05" not in d:
        # We have not spatial convergence data
        continue

    for key1, d1 in d.items():
        if "dt0.05" not in key1:
            continue  # Only select data with "dt0.05"
        for key, value in d1.items():
            results[key].append(value)

df = pd.DataFrame(results)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,import_time,timestamp,...,rel_max_displacement_perc_z,time_to_max_displacement_z
0,9.000000e-07,2023-04-12 20:33:25.203391,...,0.486886,0.05
1,1.001000e-06,2023-04-12 20:33:25.502552,...,0.488458,0.05
2,1.000000e-06,2023-04-12 20:33:32.537703,...,0.488168,0.05
3,1.100000e-06,2023-04-12 17:17:24.396818,...,0.486886,0.05
4,1.000000e-06,2023-04-12 17:17:24.948740,...,0.488458,0.05
5,1.000000e-06,2023-04-12 17:17:26.820219,...,0.488168,0.05
6,1.300000e-06,2023-04-12 09:53:34.139318,...,0.486886,0.05
7,8.000000e-07,2023-04-12 09:53:30.436212,...,0.488458,0.05
8,1.000000e-06,2023-04-12 09:53:31.010237,...,0.488168,0.05


In [44]:
df["timestamp"]
df.keys()
df["dx"].unique()

array([0.2, 0.1, 0.4])

In [45]:
def get_ylim(values):
    if np.isclose(values, 0).all():
        return (-0.01, 0.01)
    y_mean = np.mean(values)
    y_max = max(np.max(values), y_mean + 0.01 * abs(y_mean))
    y_min = min(np.min(values), y_mean - 0.01 * abs(y_mean))
    return (y_min, y_max)


In [48]:
columns = [c for c in df.columns if c not in ["timestamp", "simcardems_version", "sha", "dx", "dt"]]
git_hash = df["sha"]
versions = df["simcardems_version"]
dxs = df["dx"].unique()
dates = [t.date() for t in df["timestamp"]]


    
fig = make_subplots(rows=len(columns), cols=1, subplot_titles=columns, shared_xaxes=True)

yranges = []
colors = ['red', 'green', 'blue']
for color, dx in zip(colors, sorted(dxs)):
    df_dx = df[df["dx"] == dx]
    df_dx.sort_values(by='timestamp')
    
    git_hash = df_dx["sha"]
    versions = df_dx["simcardems_version"]
    dates = [t.date() for t in df_dx["timestamp"]]
    
    text = []
    for h,v, t in zip(git_hash, versions, dates):
        text.append("\n".join([
            f"<br>Git Hash: {h}</br>",
            f"<br>Version: {v}</br>",
            f"<br>Timestamp {t}</br>",
        ]))
 
    for i, c in enumerate(columns):
    
        row = i + 1
        col = 1
        showlegend=i==0
        fig.add_trace(
            go.Scatter(
                x=df_dx["timestamp"], 
                y=df_dx[c], 
                text=text, 
                hovertemplate="%{text}",
                name=f"dx={dx}", 
                legendgroup=str(dx),
                showlegend=i==0,
                marker_color=color
            ),
            row=row, col=col
        )
        fig.update_yaxes(row=row, col=col)

fig.update_layout(
    height=5000,
    showlegend=True
)
fig.show()